In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_excel('cleaned_labeled_dataset.xlsx')
df

,text,label,clean_text
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,kunjungan prabowo meresmikan menyerahkan proye...
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,anies tepuk tangan meriah rektor mewajibkan ma...
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,memang bener memang pendukung goblok pendukung...
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,anies bersikap kritis kinerja prabowo dianggap...
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,anies baswedan harap asn tni polri pegang sump...
...,...,...,...
4279,RT @l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwS...,Politik,ngeliat debat kemaren prabowo kicep kekira kel...
4280,Masyarakat yakin bahwa Prabowo-Gibran memiliki...,Politik,masyarakat prabowo gibran memiliki visi sejala...
4281,imo both are irrational but yg satu jauh lebih...,Ekonomi,in my opinion both are irrational but irration...
4282,@cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM= ...,Pertahanan dan Keamanan,look at that ganjar berkecimpung lgislatif eks...


In [ ]:
null_index = df[df.isnull().any(axis=1)].index
df.loc[null_index]


,text,label,clean_text
72,@+0HCSSnYXfZ/WiNzDKgSO08o/BKbymxlncndey4/N+E= ...,Sosial Budaya,NaN


In [ ]:
#drop index 72
df = df.drop(null_index)

In [ ]:
X = df['clean_text'].values
y = df['label'].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((3426,), (857,), (3426,), (857,))

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
# max length of the text
length = 0
for text in X_train:
    if len(text.split()) > length:
        length = len(text.split())
length


337

In [ ]:
# tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=length, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
vocab_size = len(word_index)
vocab_size

11714

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_seq[0]

[1, 1, 1, 1, 1, 4, 1]

In [ ]:
# padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train_seq, maxlen=length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=length, padding='post', truncating='post')
X_train_pad.shape, X_test_pad.shape

((3426, 337), (857, 337))

In [38]:
# model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D, Dropout

model = Sequential([
    Embedding(vocab_size+1, 64, input_length=length),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(8, activation='softmax')
])


In [39]:
import tensorflow as tf

from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[f1_score])

In [40]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=50, callbacks=[callback])


Epoch 1/50
108/108 [==============================] - 20s 119ms/step - loss: 1.2765 - f1_score: 2.2368 - val_loss: 1.2315 - val_f1_score: 2.1669
Epoch 2/50
108/108 [==============================] - 4s 40ms/step - loss: 1.2153 - f1_score: 1.3859 - val_loss: 1.1995 - val_f1_score: 1.1511
Epoch 3/50
108/108 [==============================] - 4s 33ms/step - loss: 1.2141 - f1_score: 1.3919 - val_loss: 1.2040 - val_f1_score: 1.1838
Epoch 4/50
108/108 [==============================] - 4s 34ms/step - loss: 1.2100 - f1_score: 1.3557 - val_loss: 1.1995 - val_f1_score: 1.7919
Epoch 5/50
108/108 [==============================] - 3s 32ms/step - loss: 1.2048 - f1_score: 1.3381 - val_loss: 1.2004 - val_f1_score: 1.1511
Epoch 6/50
108/108 [==============================] - 3s 28ms/step - loss: 1.2079 - f1_score: 1.3020 - val_loss: 1.1993 - val_f1_score: 1.1184
Epoch 7/50
108/108 [==============================] - 3s 29ms/step - loss: 1.2083 - f1_score: 1.3614 - val_loss: 1.2087 - val_f1_score: 1.85

In [41]:
# evaluasi balance accuracy
from sklearn.metrics import balanced_accuracy_score
import numpy as np

y_pred = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)
balanced_accuracy_score(y_test, y_pred_classes)

27/27 [==============================] - 1s 9ms/step


0.125

In [42]:
# seimbangkan data dengan class weight
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
class_weights

{0: 9.309782608695652,
 1: 1.9644495412844036,
 2: 28.55,
 3: 2.078883495145631,
 4: 2.01056338028169,
 5: 0.18635770234986945,
 6: 1.3509463722397477,
 7: 3.7898230088495577}

In [43]:
history2 = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=50, callbacks=[callback], class_weight=class_weights)


Epoch 1/50
108/108 [==============================] - 6s 29ms/step - loss: 2.1526 - f1_score: 11.8967 - val_loss: 2.0192 - val_f1_score: 13.3691
Epoch 2/50
108/108 [==============================] - 3s 25ms/step - loss: 2.0850 - f1_score: 13.3552 - val_loss: 2.0461 - val_f1_score: 13.3691
Epoch 3/50
108/108 [==============================] - 3s 29ms/step - loss: 2.0818 - f1_score: 13.4144 - val_loss: 2.0715 - val_f1_score: 13.4966
Epoch 4/50
108/108 [==============================] - 3s 31ms/step - loss: 2.0833 - f1_score: 13.4134 - val_loss: 2.0747 - val_f1_score: 13.4966


In [44]:
y_pred = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)
balanced_accuracy_score(y_test, y_pred_classes)

27/27 [==============================] - 0s 13ms/step


0.125

In [45]:
# sampling data
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train_pad, y_train)
X_train_ros.shape, y_train_ros.shape



((18384, 337), (18384,))

In [46]:

# training
history3 = model.fit(X_train_ros, y_train_ros, validation_data=(X_test_pad, y_test), epochs=50, callbacks=[callback])



Epoch 1/50
575/575 [==============================] - 23s 41ms/step - loss: 2.0797 - f1_score: 8.0524 - val_loss: 2.0648 - val_f1_score: 13.2416
Epoch 2/50
575/575 [==============================] - 14s 25ms/step - loss: 2.0797 - f1_score: 8.0611 - val_loss: 2.0861 - val_f1_score: 13.3691
Epoch 3/50
575/575 [==============================] - 15s 26ms/step - loss: 2.0797 - f1_score: 8.1142 - val_loss: 2.0937 - val_f1_score: 13.4966
Epoch 4/50
575/575 [==============================] - 15s 27ms/step - loss: 2.0797 - f1_score: 7.9897 - val_loss: 2.0724 - val_f1_score: 13.3691


In [47]:
# evaluasi balance accuracy
y_pred = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)
balanced_accuracy_score(y_test, y_pred_classes)

27/27 [==============================] - 0s 15ms/step


0.125

In [48]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

!unzip glove.6B.zip

--2024-06-15 15:46:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-06-15 15:46:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-06-15 15:46:11--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [49]:
# train dengan pre-trained model
from tensorflow.keras.models import load_model

embeddings_index = {}

with open('glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embeddings_index['the']



array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

In [50]:
embedding_matrix = np.zeros((vocab_size+1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector



In [51]:
model2 = Sequential([
    Embedding(vocab_size+1, 100, weights=[embedding_matrix], input_length=length, trainable=False),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(8, activation='softmax')
])

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[f1_score])


In [52]:

history4 = model2.fit(X_train_ros, y_train_ros, validation_data=(X_test_pad, y_test), epochs=50, callbacks=[callback])

Epoch 1/50
575/575 [==============================] - 28s 41ms/step - loss: 2.0797 - f1_score: 7.9811 - val_loss: 2.0819 - val_f1_score: 13.3691
Epoch 2/50
575/575 [==============================] - 14s 25ms/step - loss: 2.0780 - f1_score: 7.9347 - val_loss: 2.0726 - val_f1_score: 13.3714
Epoch 3/50
575/575 [==============================] - 20s 35ms/step - loss: 2.0779 - f1_score: 8.1008 - val_loss: 2.0764 - val_f1_score: 13.3691
Epoch 4/50
575/575 [==============================] - 15s 27ms/step - loss: 2.0779 - f1_score: 7.9860 - val_loss: 2.0842 - val_f1_score: 13.4966
Epoch 5/50
575/575 [==============================] - 17s 29ms/step - loss: 2.0776 - f1_score: 8.0322 - val_loss: 2.0802 - val_f1_score: 13.3691


In [53]:
y_pred = model2.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)
balanced_accuracy_score(y_test, y_pred_classes)

27/27 [==============================] - 1s 16ms/step


0.125